<a href="https://colab.research.google.com/github/phutaekwondo/CS114.L21/blob/main/duDoanDiemCuoiKy_homeWork/draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

># **Data Discovery**

---

##**Nhóm thực hiện**
1. Lại Nguyễn Vĩnh Phú - 19522020
2. Nguyễn Chí Cường - 19521299
3. Trương Thế Trương - 19522447

##***Dữ liệu***

1. [We code](https://github.com/phutaekwondo/CS114.L21/blob/main/duDoanDiemCuoiKy_homeWork/wecode.csv)
2. [score training](https://github.com/phutaekwondo/CS114.L21/blob/main/duDoanDiemCuoiKy_homeWork/score.csv)

In [12]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
!git clone https://github.com/phutaekwondo/CS114.L21

Cloning into 'CS114.L21'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 144 (delta 51), reused 36 (delta 9), pack-reused 0
Receiving objects: 100% (144/144), 9.08 MiB | 13.45 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [3]:
url_wecode = '/content/CS114.L21/duDoanDiemCuoiKy_homeWork/wecode.csv'
url_score = '/content/CS114.L21/duDoanDiemCuoiKy_homeWork/score.csv'

In [4]:
data_wecode = pd.read_csv(url_wecode)
data_score = pd.read_csv(url_score)

In [5]:
data_wecode

,id,assignment_id,problem_id,user_id,is_final,status,pre_score,coefficient,file_name,language_id,created_at,updated_at
0,4,0,3,4,1,SCORE,10000,100,solution-0,2,2020-10-30 10:58:11,2020-10-30 10:58:19
1,5,0,4,4,1,SCORE,10000,100,solution-1,2,2020-10-30 11:07:09,2020-10-30 11:07:17
2,6,0,5,4,0,Compilation Error,0,100,solution-2,2,2020-10-30 11:08:01,2020-10-30 11:23:36
3,7,0,5,4,0,Compilation Error,0,100,solution-3,2,2020-10-30 11:11:41,2020-10-30 11:11:42
4,8,0,5,4,0,Compilation Error,0,100,solution-4,2,2020-10-30 11:15:06,2020-10-30 11:15:07
...,...,...,...,...,...,...,...,...,...,...,...,...
264601,266667,0,25,1796,0,SCORE,0,100,solution-3977,4,2021-04-28 9:34:20,2021-04-28 9:34:27
264602,266668,0,25,1796,0,Compilation Error,0,100,solution-3978,2,2021-04-28 9:35:07,2021-04-28 9:35:08
264603,266669,0,25,1796,1,SCORE,10000,100,solution-3979,4,2021-04-28 9:35:15,2021-04-28 9:35:21
264604,266670,0,150,255,1,SCORE,6250,100,solution-3980,2,2021-05-11 1:34:57,2021-05-11 1:35:28


##Khám phá mục score

In [ ]:
discove_score = {'is_final': data_wecode['is_final'], 'pre_score': data_wecode['pre_score']}
discove_score = pd.DataFrame(discove_score, columns=['is_final', 'pre_score'])

In [ ]:
count_score = Counter(zip(discove_score['is_final'],discove_score['pre_score'])).items()
show_score = pd.DataFrame(count_score, columns=['x', 'y'])
show_score

,x,y
0,"(1, 10000)",95332
1,"(0, 0)",79796
2,"(0, 8000)",7976
3,"(0, 10000)",4508
4,"(1, 0)",2726
...,...,...
656,"(0, 3583)",1
657,"(1, 7250)",1
658,"(0, 5583)",1
659,"(0, 5666)",1


##Tạo dataframe số lượng user_id kết với problem_id

features:

+ số lần nộp bài để pass(theo problem_id, user_id )
+ time last submit - time first submit ( theo pro_id, user_id)
+ max (pre_score) (theo pro_id, user_id)
+ is_final = max(data_wecode['is_final']

features table

user_id | problem_id | số lần nộp | is_final | max(pre_score)|time|
--------|------------|------------|------|----------------|--------|

In [ ]:
first_feature = {'user_id': data_wecode['user_id'], 'problem_id': data_wecode['problem_id']}
first_feature = pd.DataFrame(first_feature, columns = ['user_id', 'problem_id'])

#Đếm submit theo user_id và problem_id
count_submit = Counter(zip(first_feature['user_id'], first_feature['problem_id'])).items()
user_feature = pd.DataFrame(count_submit, columns=['user_id, problem_id', 'amount_submit'])#lúc này có user_id và problem_id chung 1 column

#sắp xếp lại data - tách user_id và problem_id
temp = pd.DataFrame(user_feature['user_id, problem_id'].tolist(), columns = ['user_id','problem_id']) #tạo temp = 2 cột user_id và problem_id khi tác ra
del user_feature['user_id, problem_id'] #Xóa cột
user_feature['user_id'], user_feature['problem_id'] = temp['user_id'], temp['problem_id']#thêm lại 2 cột
cols = user_feature.columns.tolist()#sếp xếp lại
cols = cols[-2:] + cols[:1]
user_feature = user_feature[cols]
#sort theo user_id_problem_id và set index
user_feature = user_feature.sort_values(by=['user_id', 'problem_id']).set_index(pd.Index(range(len(user_feature))))

>Lấy max điểm, time, is_final cho từng user_problem_id

In [ ]:
other_feature_user = {'user_problem_id': pd.Series(zip(data_wecode['user_id'], data_wecode['problem_id'])), 'is_final': data_wecode['is_final'],
                      'pre_score': data_wecode['pre_score'], 'time': data_wecode['updated_at']}
other_feature_user = pd.DataFrame(other_feature_user, columns=['user_problem_id', 'is_final', 'pre_score', 'time'])
other_feature_user = other_feature_user.sort_values(by='user_problem_id').set_index(pd.Index(range(len(other_feature_user))))
other_feature = other_feature_user.groupby(['user_problem_id'], as_index=False).agg('max')

In [ ]:
#Đưa cột pre_score vào user_feature. Vì 2 bảng sort giống nhau nên việc thêm vào sẽ ko sai vị trí
user_feature['is_final'], user_feature['pre_score'], user_feature['time'] = other_feature['is_final'], other_feature['pre_score'], other_feature['time']
user_feature

,user_id,problem_id,amount_submit,is_final,pre_score,time
0,1,81,1,1,0,2020-11-08 16:50:50
1,1,370,1,1,10000,2020-12-01 2:00:05
2,4,3,1,1,10000,2020-10-30 10:58:19
3,4,4,1,1,10000,2020-10-30 11:07:17
4,4,5,5,1,10000,2020-10-30 11:23:36
...,...,...,...,...,...,...
97185,1868,637,4,1,10000,2021-01-22 15:23:21
97186,1868,638,4,1,10000,2021-01-22 15:38:20
97187,1868,639,2,1,10000,2021-01-22 15:44:30
97188,1868,640,2,1,10000,2021-01-22 15:49:31


In [ ]:
#Lưu bằng vào driver-> chủ yếu khỏi chạy lại đoạn code trên vì code trên chạy lâu
from google.colab import drive
drive.mount('/content/drive')
with open('/content/drive/MyDrive/Colab Notebooks/user_feature.csv', 'w') as f:
  user_feature.to_csv(f, index=False)

##Continute

In [6]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
user_feature = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/user_feature.csv')
user_feature

,user_id,problem_id,amount_submit,is_final,pre_score,time
0,1,81,1,1,0,2020-11-08 16:50:50
1,1,370,1,1,10000,2020-12-01 2:00:05
2,4,3,1,1,10000,2020-10-30 10:58:19
3,4,4,1,1,10000,2020-10-30 11:07:17
4,4,5,5,1,10000,2020-10-30 11:23:36
...,...,...,...,...,...,...
97185,1868,637,4,1,10000,2021-01-22 15:23:21
97186,1868,638,4,1,10000,2021-01-22 15:38:20
97187,1868,639,2,1,10000,2021-01-22 15:44:30
97188,1868,640,2,1,10000,2021-01-22 15:49:31


##Xử lý user_features

> Biến đổi về theo user_id để có thể ánh xạ với feature_score


In [20]:
synthetic = user_feature.groupby(['user_id'], as_index=False).agg(list)

In [21]:
synthetic

,user_id,problem_id,amount_submit,is_final,pre_score,time
0,1,"[81, 370]","[1, 1]","[1, 1]","[0, 10000]","[2020-11-08 16:50:50, 2020-12-01 2:00:05]"
1,4,"[3, 4, 5, 6, 7, 8, 9, 81, 101, 104, 106, 107, ...","[1, 1, 5, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10000, 10000, 10000, 10000, 10000, 10000, 100...","[2020-10-30 10:58:19, 2020-10-30 11:07:17, 202..."
2,5,"[1, 4, 81, 176, 243]","[1, 1, 1, 1, 2]","[1, 1, 1, 1, 1]","[10000, 10000, 0, 10000, 0]","[2020-11-20 3:01:22, 2020-11-20 8:58:15, 2020-..."
3,6,"[1, 3, 9, 80, 81, 82, 83, 84, 85, 86, 87, 88, ...","[2, 2, 1, 5, 2, 1, 4, 1, 1, 1, 4, 3, 3, 2, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 10000, 0, 10000, 10000, 10000, 10000, 1000...","[2020-11-05 3:30:52, 2020-11-05 5:22:57, 2020-..."
4,7,"[265, 390]","[2, 3]","[1, 1]","[8571, 638]","[2020-12-26 3:38:17, 2020-12-30 3:26:50]"
...,...,...,...,...,...,...
1154,1860,"[3, 4, 5, 7, 8, 9, 11, 25, 174, 175, 176, 177,...","[1, 2, 2, 3, 2, 1, 2, 2, 1, 2, 2, 4, 2, 1, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10000, 0, 10000, 0, 10000, 0, 0, 10000, 0, 10...","[2020-11-27 7:58:08, 2020-11-27 8:05:30, 2020-..."
1155,1862,"[1, 10, 11, 176, 181, 182]","[4, 2, 3, 1, 1, 1]","[1, 1, 1, 1, 1, 1]","[10000, 0, 0, 10000, 10000, 10000]","[2020-11-27 15:12:08, 2020-11-27 15:16:28, 202..."
1156,1864,"[1, 3, 4, 5, 6, 7, 8, 10, 11, 80, 81, 82, 84, ...","[5, 6, 10, 1, 3, 5, 5, 3, 12, 4, 5, 3, 2, 1, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10000, 10000, 10000, 10000, 0, 10000, 10000, ...","[2020-12-15 7:31:30, 2020-12-15 7:51:11, 2020-..."
1157,1865,"[343, 344, 345, 346, 347, 348, 349, 352, 353, ...","[6, 5, 10, 9, 8, 12, 5, 6, 1, 3, 7, 7, 3, 2, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1000, 1000, 1000, 10000, 10000, 10000, 10000,...","[2021-01-12 15:42:25, 2021-01-12 15:26:02, 202..."


In [13]:
count_sb_problem = Counter(data_wecode['problem_id']).items()
score_table = pd.DataFrame(count_sb_problem, columns=['problem_id', 'amount_submit'])
max_submit = max(score_table['amount_submit'])
score_table['score'] = score_table['amount_submit'] * 10 / max_submit
score_table = score_table.set_index('problem_id')

In [14]:
score_table

,amount_submit,score
problem_id,,
3,2992,6.322908
4,4213,8.903212
5,1271,2.685968
6,4417,9.334320
7,1821,3.848267
...,...,...
692,5,0.010566
697,23,0.048605
698,17,0.035926


#**Score Data**

In [28]:
features_score = {'id': data_score['id'], 'CK': data_score['CK']}
features_score = pd.DataFrame(features_score, columns = ['id', 'CK'])

In [29]:
features_score

,id,CK
0,22,8.5
1,24,9.5
2,27,10
3,28,9.5
4,29,8.5
...,...,...
970,1857,4
971,1859,1.5
972,1862,7.5
973,1865,7.5


# **Nhận Xét**